In [18]:
# Thêm thư viện
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from imutils import paths
from keras.applications import VGG16
from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
# from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.optimizers import RMSprop
from keras.applications import VGG16
from keras.layers import Input
from keras.models import Model , Sequential
# from keras.layers.core import Dense
# from keras.layers.core import Dropout
# from keras.layers.core import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
import numpy as np
import random
import os

In [19]:
# Lấy các đường dẫn đến ảnh.
image_path = list(paths.list_images('dataset/'))

# Đổi vị trí ngẫu nhiên các đường dẫn ảnh
random.shuffle(image_path)

In [20]:
# Đường dẫn ảnh sẽ là dataset/tên_loài_hoa/tên_ảnh ví dụ dataset/Bluebell/image_0241.jpg nên p.split(os.path.sep)[-2] sẽ lấy ra được tên loài hoa
labels = [p.split(os.path.sep)[-2] for p in image_path]

# Chuyển tên các loài hoa thành số
le = LabelEncoder()
labels = le.fit_transform(labels)

# One-hot encoding
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [21]:
# Load ảnh và resize về đúng kích thước mà VGG 16 cần là (224,224)
list_image = []
for (j, imagePath) in enumerate(image_path):
    image = load_img(imagePath, target_size=(224, 224))
    image = img_to_array(image)
    
    image = np.expand_dims(image, 0)
    image = imagenet_utils.preprocess_input(image)
    
    list_image.append(image)
    
list_image = np.vstack(list_image)

In [22]:
# Load model VGG 16 của ImageNet dataset, include_top=False để bỏ phần Fully connected layer ở cuối.
baseModel = VGG16(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))

# Xây thêm các layer
# Lấy output của ConvNet trong VGG16
fcHead = baseModel.output

# Flatten trước khi dùng FCs
fcHead = Flatten(name='flatten')(fcHead)

# Thêm FC
fcHead = Dense(256, activation='relu')(fcHead)
fcHead = Dropout(0.5)(fcHead)

# Output layer với softmax activation
fcHead = Dense(17, activation='softmax')(fcHead)

# Xây dựng model bằng việc nối ConvNet của VGG16 và fcHead
model = Model(inputs=baseModel.input, outputs=fcHead)

In [23]:
# Chia traing set, test set tỉ lệ 80-20
X_train, X_test, y_train, y_test = train_test_split(list_image, labels, test_size=0.2, random_state=42)

In [24]:
# augmentation cho training data
aug_train = ImageDataGenerator(rescale=1./255, rotation_range=30, width_shift_range=0.1, height_shift_range=0.1, shear_range=0.2, 
                         zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')
# augementation cho test
aug_test= ImageDataGenerator(rescale=1./255)

In [28]:
# freeze VGG model
for layer in baseModel.layers:
    layer.trainable = False
    
opt = RMSprop(0.001)
# model.compile(opt, 'categorical_crossentropy', ['accuracy'])
model.compile(opt, 'categorical_crossentropy', metrics=['accuracy'])
numOfEpoch = 25
H = model.fit(aug_train.flow(X_train, y_train, batch_size=32), # thay fit_generator bằng fit (version mới của keras)
              steps_per_epoch=len(X_train)//32,
              validation_data=(aug_test.flow(X_test, y_test, batch_size=32)),
              validation_steps=len(X_test)//32,
              epochs=numOfEpoch)


Epoch 1/25


c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


34/34 ━━━━━━━━━━━━━━━━━━━━ 85s 2s/step - accuracy: 0.0945 - loss: 6.6581 - val_accuracy: 0.3281 - val_loss: 2.3185
Epoch 2/25


c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.5000 - val_loss: 1.0439
Epoch 3/25
34/34 ━━━━━━━━━━━━━━━━━━━━ 83s 2s/step - accuracy: 0.2190 - loss: 2.5841 - val_accuracy: 0.3750 - val_loss: 1.8124
Epoch 4/25
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.3125 - val_loss: 0.9941
Epoch 5/25
34/34 ━━━━━━━━━━━━━━━━━━━━ 95s 3s/step - accuracy: 0.3098 - loss: 2.1503 - val_accuracy: 0.4688 - val_loss: 1.7191
Epoch 6/25
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.8125 - val_loss: 0.5456
Epoch 7/25
34/34 ━━━━━━━━━━━━━━━━━━━━ 89s 3s/step - accuracy: 0.3858 - loss: 1.8803 - val_accuracy: 0.6289 - val_loss: 1.2970
Epoch 8/25
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.8125 - val_loss: 0.7028
Epoch 9/25
34/34 ━━━━━━━━━━━━━━━━━━━━ 88s 3s/step - accuracy: 0.4494 - loss: 1.6731 - val_acc

In [31]:
# unfreeze some last CNN layer:
for layer in baseModel.layers[15:]:
    layer.trainable = True

numOfEpoch = 35
opt = SGD(0.001)
model.compile(opt, 'categorical_crossentropy', metrics=['accuracy'])
H = model.fit(aug_train.flow(X_train, y_train, batch_size=32), 
                        steps_per_epoch=len(X_train)//32,
                        validation_data=(aug_test.flow(X_test, y_test, batch_size=32)),
                        validation_steps=len(X_test)//32,
                        epochs=numOfEpoch)

Epoch 1/35
34/34 ━━━━━━━━━━━━━━━━━━━━ 86s 2s/step - accuracy: 0.7424 - loss: 0.7270 - val_accuracy: 0.9141 - val_loss: 0.3100
Epoch 2/35
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0904
Epoch 3/35
34/34 ━━━━━━━━━━━━━━━━━━━━ 83s 2s/step - accuracy: 0.7704 - loss: 0.6951 - val_accuracy: 0.9297 - val_loss: 0.2887
Epoch 4/35
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.9375 - val_loss: 0.1045
Epoch 5/35
34/34 ━━━━━━━━━━━━━━━━━━━━ 82s 2s/step - accuracy: 0.7550 - loss: 0.6655 - val_accuracy: 0.9297 - val_loss: 0.2691
Epoch 6/35
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.9375 - val_loss: 0.1338
Epoch 7/35
34/34 ━━━━━━━━━━━━━━━━━━━━ 89s 3s/step - accuracy: 0.7862 - loss: 0.6419 - val_accuracy: 0.9062 - val_loss: 0.2836
Epoch 8/35
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+0

In [32]:
model.save('Model_VGG16_fine_tune.h5')